In [3]:
#==================================================================
#Program: WSI-Classifier-Reimplementation-Tyrell
#Version: 1.4
#Author: David Helminiak
#Date Created: 4 April 2025
#Date Last Modified: 12 September 2025
#Description: Classify WSI .jpg files placed inside ./WSI/ folder
#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#Load tracer for debugging
from IPython.core.debugger import set_trace as Tracer

#Configuration settings that do not generally need to be changed
#====================================================================

#Which channel should be used to assess whether patches are in the foreground: 'red', 'green', 'gray' (default: 'red')
#'gray' used for original work replication
channel_extraction = 'red'

#What threshold should be used for patch extraction; minimum allowable % of chosen-channel (channel_extraction) values >= backgroundLevel (default: 0.2)
#0.8 used for original work replication
thresholdPatch_extraction = 0.2

#Minimum chosen channel value [0, 255] for a location to contribute to the backgroundThreshold criteria (default: 5)
backgroundLevel = 5

#What ratio of malignant to benign patches should be used to predict a whole WSI as malignant (default: 0.15)
#When set to 0, will label a WSI as malignant if any one patch is predicted as malignant
#Unknown what the original work used for this value, but for original work replication, a value of 0.15 seems appropriate 
#Replication of original results occurs with values between 0.12-0.19
thresholdWSI_prediction = 0.15

#How many samples should be submitted in a batch through pytorch models used in classifier
#Incrementing in powers of 2 recommended to best leverage common GPU hardware designs
#For ResNet and DenseNet a 2080TI 11GB can handle 64x3x224x224 (resizeSize=224) or 16x3x400x400 (resizeSize=0)
#In some cases, the GPU memory may not be fully released (even when explicitly told to do so), so using a lower batch size may help prevent OOM
batchsizeClassifier = 32

#Patch size for WSI-padding/extraction/classification
patchSize = 400

#Specify what symmetrical dimension patches should be resized to; if no resizing is desired leave as 0 (default: 224)
#Leaving as 0 will increase training time (also must change batchsizeClassifier), but can lead to improved scores
#Original implementation uses 224, though with adaptive average pooling this isn't actually neccessary
resizeSize_patches = 224

#Specify what symmetrical dimension WSI should be resized to when generating saliency maps (default: 224)
#If fusion method were to be further developed, this should be changed to maintain the original sample aspect ratio. 
resizeSize_WSI = 224

#How thick should the grid lines be when generating overlay images (defualt: 50)
gridThickness = 50

#Which pre-trained weight sets should be used for ResNet50 model: 'IMAGENET1K_V1', 'IMAGENET1K_V2'
#Unclear which weights were used for TensorFlow ResNet50 variant in original classifier implementation
#V2 did improve scores a bit in RANDS when using resizeSize = 0
weightsResNet = 'IMAGENET1K_V2'

#Which pre-trained weight sets should be used for DenseNet model: 'IMAGENET1K_V1'
weightsDenseNet = 'IMAGENET1K_V1'

#What weight should be used when overlaying data
overlayWeight = 0.5

#Which GPU(s) devices should be used (last specified used if model is not multi-GPU capable)
#default: [-1], any/all available; CPU only: [])
gpus = [-1]

#Should parallelization calls be used to leverage multithreading where able
#Not currently used in this program
parallelization = True

#If parallelization is enabled, how many CPU threads should be used? (0 will use any/all available)
#Recommend starting at half of the available system threads if using hyperthreading,
#or 1-2 less than the number of system CPU cores if not using hyperthreading.
#Adjust to where the CPU just below 100% usage during parallel operations 
availableThreads = 0

#Seed vale for repeatability
manualSeedValue = 0




#Imports 
#================================================================
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import copy
import cupy as cp
import cv2
import glob
import matplotlib
import natsort
import numpy as np
import os
import pandas as pd
import psutil
import shutil
import random
import time
import torch

from matplotlib import colors
from matplotlib import pyplot as plt
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision import transforms
from torchvision.transforms import v2
from xgboost import XGBClassifier

#Enivronment 
#================================================================

#Setup deterministic behavior for torch (this alone does not affect CUDA-specific operations)
if manualSeedValue != -1: torch.use_deterministic_algorithms(True, warn_only=False)

#Detect logical and physical core counts, determining if hyperthreading is active
logicalCountCPU = psutil.cpu_count(logical=True)
physicalCountCPU = psutil.cpu_count(logical=False)
hyperthreading = logicalCountCPU > physicalCountCPU

#Set parallel CPU usage limit, disabling if there is only one thread remaining
#Ray documentation indicates num_cpus should be out of the number of logical cores/threads
#In practice, specifying a number closer to, or just below, the count of physical cores maximizes performance
#Any ray.remote calls need to specify num_cpus to set environmental OMP_NUM_THREADS variable correctly
if parallelization: 
    if availableThreads==0: numberCPUS = physicalCountCPU
    else: numberCPUS = availableThreads
    if numberCPUS <= 1: parallelization = False
if not parallelization: numberCPUS = 1

#Determine available GPUs 
if not torch.cuda.is_available(): gpus = []
if (len(gpus) > 0) and (gpus[0] == -1): gpus = [*range(torch.cuda.device_count())]

    
#Internal variables 
#================================================================
    
#Define matplotlib colors for visualization
cmapClasses = colors.ListedColormap(['lime', 'red', 'black'])

#Determine offset to avoid overlapping squares in grid visualization
gridThicknessOffset = gridThickness//2


#Class and function definitions
#================================================================

#Define how to reset the random seed for deterministic repeatable RNG
def resetRandom():
    if manualSeedValue != -1:
        torch.manual_seed(manualSeedValue)
        torch.cuda.manual_seed_all(manualSeedValue)
        np.random.seed(manualSeedValue)
        random.seed(manualSeedValue)

#OpenCV does not output sharp corners with its rectangle method...unless it's filled in
def rectangle(image, startPos, endPos, color):
    image = cv2.rectangle(image, startPos, endPos, color, -1)
    image = cv2.rectangle(image, (startPos[0]+gridThicknessOffset, startPos[1]+gridThicknessOffset), (endPos[0]-gridThicknessOffset, endPos[1]-gridThicknessOffset), (0, 0, 0), -1)
    return image

#Convert numpy array to contiguous tensor; issues with lambda functions when using multiprocessing
def contiguousTensor(inputs):
    return torch.from_numpy(inputs).contiguous()

#Rescale tensor; issues with lambda functions when using multiprocessing
def rescaleTensor(inputs):
    return inputs.to(dtype=torch.get_default_dtype()).div(255)

#Generate a torch transform needed for preprocessing image data
def generateTransform(resizeSize=[], rescale=False, normalize=False):
    transform = [contiguousTensor]
    if len(resizeSize) > 0: transform.append(v2.Resize(tuple(resizeSize)))
    if rescale: transform.append(rescaleTensor)
    if normalize: transform.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return transforms.Compose(transform)

#Load and preprocess data image files
#ToTensor() swaps axes (so not used); convert to contiguous torch tensor manually
#Rescaling and changing data type only after resizing (otherwise can escape [0, 1])
class DataPreprocessing_Classifier(Dataset):
    def __init__(self, filenames, resizeSize):
        super().__init__()
        self.filenames = filenames
        self.numFiles = len(self.filenames)
        if resizeSize > 0: self.transform = generateTransform([resizeSize, resizeSize], True, True)
        else: self.transform = generateTransform([], True, True)
        
    #Rearranging import dimensions, allows resize transform before tensor-conversion/rescaling; preserves precision and output data range
    def __getitem__(self, index): return self.transform(np.moveaxis(cv2.cvtColor(cv2.imread(self.filenames[index], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB), -1, 0))
    
    def __len__(self): return self.numFiles

def get_total_space(pred_map):
    pred_map = pred_map.astype(np.uint8)
    pred_background = (pred_map.shape[0] * pred_map.shape[1]) - np.count_nonzero(pred_map)

    connectivity = 8
    output = cv2.connectedComponentsWithStats(pred_map, connectivity, cv2.CV_32S)
    num_labels, _, stats, _ = output

    return sum([stats[num, cv2.CC_STAT_AREA] for num in range(num_labels) if stats[num, cv2.CC_STAT_AREA] != pred_background])

    
#Create/load/run an available XGBClassifier model with ResNet50 feature extraction and GradCam++ using DenseNet169
class Classifier_XGB_RN50_GCPP_DN169():
    
    def __init__(self, modelFile): 
        
        #Set internal reference to model file
        self.modelFile = modelFile
        
        #Setup available computation environment
        self.device = f"cuda:{gpus[-1]}" if len(gpus) > 0 else "cpu"
        self.torchDevice = torch.device(self.device)
        
        #Set default cuda device for XGBClassifier input data
        if len(gpus) > 0: cp.cuda.Device(gpus[-1]).use()
    
    def classifyWSI(self, sampleName, imageWSI, patchLocations, patchNames, patchFilenames, dir_Results_Classification, dir_Results_Patches):
        
        #Start timer for data preparation
        timePrepareStart = time.perf_counter()
        
        #Store needed input variables internally
        self.sampleName = sampleName
        self.imageWSI = imageWSI
        self.patchLocations = patchLocations
        self.dir_Results_Classification = dir_Results_Classification
        
        #Prepare data objects for obtaining/processing PyTorch model inputs
        self.patchDataloader = DataLoader(DataPreprocessing_Classifier(patchFilenames, resizeSize_patches), batch_size=batchsizeClassifier, num_workers=0, shuffle=False, pin_memory=True)
        if resizeSize_WSI > 0: transform = generateTransform([resizeSize_WSI, resizeSize_WSI], True, True)
        else: transform = generateTransform([], True, True)
        self.WSIDataloader = WSIDataloader = [item for item in DataLoader(transform(np.expand_dims(np.moveaxis(imageWSI, -1, 0), 0).copy()), batch_size=1, num_workers=0, shuffle=False, pin_memory=True)][0]
        
        #Log classifier preparation time
        timePrepareDiff = time.perf_counter()-timePrepareStart
        
        #Start classification timer
        timeClassifyStart = time.perf_counter()
        
        #Compute patch weights from DenseNet169/GradCam++ saliency map for fusion mechanism
        self.computePatchWeights()

        #Compute features for each patch using ResNet50
        self.computeFeatures()
        
        #Classify patches
        self.classifyPatches()
        
        #Classify WSI according ratio of malignant to foreground patches
        if thresholdWSI_prediction == 0:
            if np.sum(self.predictionsFusion) > 0: self.predictionFusionWSI = 1
            else: self.predictionFusionWSI = 0
        else: 
            if np.mean(self.predictionsFusion) >= thresholdWSI_prediction: self.predictionFusionWSI = 1
            else: self.predictionFusionWSI = 0
        
        #Log classification time
        timeClassifyDiff = time.perf_counter()-timeClassifyStart
        
        #Start timer for saving results to disk
        timeSaveResultsStart = time.perf_counter()
        
        #Save patch predictions to disk
        dataPrintout, dataPrintoutNames = [patchNames, self.predictions, self.predictionsFusion], ['Names', 'Predictions', 'Fusion Predictions']
        dataPrintout = pd.DataFrame(np.asarray(dataPrintout)).transpose()
        dataPrintout.columns=dataPrintoutNames
        dataPrintout.to_csv(self.dir_Results_Classification + 'predictions_Patches.csv', index=False)
        
        #Save WSI predictions to disk
        dataPrintout, dataPrintoutNames = [self.sampleName, self.predictionFusionWSI], ['Names', 'Fusion Prediction']
        dataPrintout = pd.DataFrame(np.asarray(dataPrintout)).transpose()
        dataPrintout.columns=dataPrintoutNames
        dataPrintout.to_csv(self.dir_Results_Classification + 'predictions_WSI.csv', index=False)
        
        #Log results saving time 
        timeSaveResultsDiff = time.perf_counter()-timeSaveResultsStart
        
        #Start timer for visualization generation
        timeVisualizationStart = time.perf_counter()
        
        #Create grid overlays
        gridOverlay_Predictions = np.zeros(self.imageWSI.shape, dtype=np.uint8)
        colorsPredictions = (cmapClasses(self.predictions)[:,:3].astype(np.uint8)*255).tolist()
        gridOverlay_PredictionsFusion = np.zeros(self.imageWSI.shape, dtype=np.uint8)
        colorsFusion = (cmapClasses(self.predictionsFusion)[:,:3].astype(np.uint8)*255).tolist()
        for patchIndex in range(0, len(patchLocations)):
            startRow, startColumn = patchLocations[patchIndex] 
            posStart, posEnd = (startColumn, startRow), (startColumn+patchSize, startRow+patchSize)
            gridOverlay_Predictions = rectangle(gridOverlay_Predictions, posStart, posEnd, colorsPredictions[patchIndex])
            gridOverlay_PredictionsFusion = rectangle(gridOverlay_PredictionsFusion, posStart, posEnd, colorsFusion[patchIndex])
        
        #Store raw overlays to disk
        filename = self.dir_Results_Classification+'gridPredictions_'+self.sampleName+'.tif'
        cv2.imwrite(filename, cv2.cvtColor(gridOverlay_Predictions, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
        filename = self.dir_Results_Classification+'gridPredictionsFusion_'+self.sampleName+'.tif'
        cv2.imwrite(filename, cv2.cvtColor(gridOverlay_PredictionsFusion, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
        
        #Overlay grids on top of WSI and store to disk
        filename = self.dir_Results_Classification+'overlaidPredictions_'+self.sampleName+'.tif'
        imageWSI_Prediction = cv2.addWeighted(self.imageWSI, 1.0, gridOverlay_Predictions, overlayWeight, 0.0)
        cv2.imwrite(filename, cv2.cvtColor(imageWSI_Prediction, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
        filename = self.dir_Results_Classification+'overlaidPredictionsFusion_'+self.sampleName+'.tif'
        imageWSI_PredictionsFusion = cv2.addWeighted(self.imageWSI, 1.0, gridOverlay_PredictionsFusion, overlayWeight, 0.0)
        cv2.imwrite(filename, cv2.cvtColor(imageWSI_PredictionsFusion, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
        
        #Log visualization time 
        timeVisualizationDiff = time.perf_counter()-timeVisualizationStart
        
        #Clear remaining internal objects that are not required after classification to save memory
        del self.sampleName, self.imageWSI, self.patchLocations, self.dir_Results_Classification, self.predictions, self.predictionsFusion
        
        return self.predictionFusionWSI, imageWSI_Prediction, imageWSI_PredictionsFusion
    
    def computeFeatures(self):
        
        #Reset RNG before setting up model; else initialization values may be inconsistent
        resetRandom()
        
        #Load pretrained ResNet50 model and set to evaluation mode
        model_ResNet = models.resnet50(weights=weightsResNet).to(self.torchDevice)
        _ = model_ResNet.train(False)

        #Extract features for each batch of sample patch images
        self.patchFeatures = []
        for data in self.patchDataloader: self.patchFeatures += model_ResNet(data.to(self.torchDevice)).detach().cpu().tolist()
        
        #Clear the ResNet model and patch dataloader
        del model_ResNet, self.patchDataloader
        if len(gpus) > 0: torch.cuda.empty_cache()
        
        #Convert list of features to an array
        self.patchFeatures = np.asarray(self.patchFeatures)
        
        #Save features to disk
        np.save(self.dir_Results_Classification + 'patchFeatures', self.patchFeatures)
    
    def computePatchWeights(self):
        
        #Reset RNG before setting up model; else initialization values may be inconsistent
        resetRandom()
            
        #Load pre-trained DenseNet
        model_DenseNet = models.densenet169(weights=weightsDenseNet)
        
        #Replace the in-built classifier; unclear how this structure and these hyperparameters were determined
        model_DenseNet.classifier = nn.Sequential(
            nn.Linear(model_DenseNet.classifier.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 2),
            nn.LogSoftmax(dim=1)
        )
        model_DenseNet = model_DenseNet.to(self.torchDevice)
        _ = model_DenseNet.train(False)
        
        #Create GradCAMPlusPlus model; see https://github.com/jacobgil/pytorch-grad-cam for additional models and options
        model_GradCamPlusPlus = GradCAMPlusPlus(model=model_DenseNet, target_layers=[model_DenseNet.features[-1]])
        
        #Compute saliency map for the WSI
        #For current GradCamPlusPlus implementation, must manually clear internal copy of the outputs from GPU cache to prevent OOM
        #Do not need to move data to device here, as managed by GradCAMPlusPlus (having already been placed on device)
        saliencyMap = model_GradCamPlusPlus(input_tensor=self.WSIDataloader, targets=None)
        del model_GradCamPlusPlus.outputs
        if len(gpus) > 0: torch.cuda.empty_cache()
        
        #Clear DenseNet, GradCamPlusPlus, and the WSI data loader
        del model_DenseNet, model_GradCamPlusPlus, self.WSIDataloader
        if len(gpus) > 0: torch.cuda.empty_cache()
        
        #Store the raw saliency map to disk
        self.saliencyMap = copy.deepcopy(saliencyMap)
        image = matplotlib.cm.jet(saliencyMap)[0,:,:,:-1].astype(np.float32)
        filename = self.dir_Results_Classification+'saliencyMap_'+self.sampleName+'.tif'
        cv2.imwrite(filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
        
        #Resize the saliency map to match the WSI dimensions
        transform = generateTransform(self.imageWSI.shape[:2], False, False)
        saliencyMap = transform(np.expand_dims(saliencyMap, 0))[0][0].numpy()
        
        #Overlay the saliency map on the WSI (grayscale for clearer visualization) and save it to disk
        overlaid = np.expand_dims(cv2.cvtColor(self.imageWSI, cv2.COLOR_RGB2GRAY), -1)
        transform = generateTransform([], True, False)
        overlaid = np.moveaxis(transform(np.moveaxis(overlaid, -1, 0)).numpy(), 0, -1)
        overlaid = show_cam_on_image(overlaid, saliencyMap, use_rgb=True, colormap=cv2.COLORMAP_HOT, image_weight=1.0-overlayWeight)
        self.overlaid = overlaid
        filename = self.dir_Results_Classification+'overlaidSaliency_'+self.sampleName+'.tif'
        cv2.imwrite(filename, cv2.cvtColor(overlaid, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
        
        #Extract saliency map data specific to sample patch locations, compute regional importance as the average value, and threshold to get weights
        self.patchWeights = []
        for locationData in self.patchLocations:
            startRow, startColumn = locationData
            patchImportance = np.mean(saliencyMap[startRow:startRow+patchSize, startColumn:startColumn+patchSize])
            if patchImportance < 0.25: self.patchWeights.append(0)
            else: self.patchWeights.append(patchImportance)
        
        #Convert list of patch weights to an array and save to disk
        self.patchWeights = np.asarray(self.patchWeights)
        np.save(self.dir_Results_Classification + 'patchWeights', self.patchWeights)
    
    #Classify patches and perform decision fusion
    def classifyPatches(self):
    
        #Load a pretrained XGBClassifier model
        model_XGBClassifier = XGBClassifier()
        model_XGBClassifier.load_model(self.modelFile)
        model_XGBClassifier._Booster.set_param({'device': self.device})
        
        #Place data on the GPU if able
        dataInput = np.asarray(self.patchFeatures.astype(np.float32))
        if len(gpus) > 0: dataInput = cp.asarray(dataInput)
        
        #Compute the raw patch predictions
        self.predictions = model_XGBClassifier.predict(dataInput)
        self.predictions = np.asarray(self.predictions)
        
        #Clear the XGBClassifier model and data on GPU
        del model_XGBClassifier, dataInput
        if len(gpus) > 0: 
            torch.cuda.empty_cache() 
            cp._default_memory_pool.free_all_blocks()
        
        #Perform decision fusion, multiplying predictions with saliency map weights (using -1 for benign and +1 for malignant)
        self.predictionsFusion = np.where(self.predictions==0, -1, 1)*self.patchWeights
        self.countSignificant = np.count_nonzero(self.predictionsFusion)
        self.predictionsFusion = np.where(self.predictionsFusion>0, 1, 0)
        del self.patchWeights

#Extract patches from WSI and save thenm
def extractPatches(imageWSI, dir_patches):

    #Split the WSI into patches and flatten
    numPatchesRow, numPatchesCol = imageWSI.shape[0]//patchSize, imageWSI.shape[1]//patchSize
    imageWSI = imageWSI.reshape(numPatchesRow, patchSize, numPatchesCol, patchSize, imageWSI.shape[2]).swapaxes(1,2)
    imageWSI = imageWSI.reshape(-1, imageWSI.shape[2], imageWSI.shape[3], imageWSI.shape[4])
    
    #Save patches to disk (always lossless) that meet a given threshold of chosen-channel values at or over a given background level
    patchLocations, patchNames, patchFilenames = [], [], []
    patchIndex = 0
    for rowNum in range(0, numPatchesRow):
        for colNum in range(0, numPatchesCol):
            image = imageWSI[patchIndex]
            if channel_extraction == 'red': channelValue = np.mean(image[:,:,0] >= backgroundLevel)
            elif channel_extraction == 'green': channelValue = np.mean(image[:,:,1] >= backgroundLevel)
            elif channel_extraction == 'gray': channelValue = np.mean(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) >= backgroundLevel)
            else: sys.exit('Error - Unknown channel selected for use during patch extraction')
            if channelValue > thresholdPatch_extraction:
                locationRow, locationColumn= rowNum*patchSize, colNum*patchSize
                patchLocations.append([locationRow, locationColumn])
                patchName = sampleName+'_'+str(patchIndex)+'_'+str(locationRow)+'_'+str(locationColumn)
                patchNames.append(patchName)
                patchFilename = dir_patches + 'PS' + patchName + '.tif'
                writeSuccess = cv2.imwrite(patchFilename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
                patchFilenames.append(patchFilename)
            patchIndex += 1

    return patchLocations, patchNames, patchFilenames






#MAIN PROGRAM
#===========================================================================
#Specify internal paths
dir_WSI = './Full_INPUT_WSI/'
dir_Results = './RESULTS/'
if not os.path.exists(dir_Results): os.mkdir(dir_Results)

#Create a new classifier object
classifier = Classifier_XGB_RN50_GCPP_DN169('./model_XGBClassifier.json')

#Classify each WSI with a .jpg extension in the 
for filename in natsort.natsorted(glob.glob(dir_WSI + '*.jpg')):

    sampleName = os.path.basename(filename).split('.')[0]
    print('Now classifying sample: ' + sampleName)

    #Create output folders for the current sample; will overwrite prior results
    dir_resultsWSI = dir_Results + sampleName
    dir_patches = dir_resultsWSI + '/Patches/'
    dir_classification = dir_resultsWSI + '/Classification/'
    if os.path.exists(dir_resultsWSI): shutil.rmtree(dir_resultsWSI)
    os.mkdir(dir_resultsWSI)
    os.mkdir(dir_patches)
    os.mkdir(dir_classification)

    #Load in the WSI (RGB ordering)
    imageWSI = cv2.cvtColor(cv2.imread(filename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)

    #Crop off the right and bottom edges for even division by the specified patch size
    numPatchesRow, numPatchesCol = imageWSI.shape[0]//patchSize, imageWSI.shape[1]//patchSize
    imageWSI = imageWSI[:numPatchesRow*patchSize, :numPatchesCol*patchSize]
    
    #Extract patches
    patchLocations, patchNames, patchFilenames = extractPatches(imageWSI, dir_patches)

    #Classify patches and WSI
    predictionFusionWSI, imageWSI_Prediction, imageWSI_PredictionsFusion = classifier.classifyWSI(sampleName, imageWSI, patchLocations, patchNames, patchFilenames, dir_classification, dir_patches)
    if predictionFusionWSI == 1: print('  Final Fusion Prediction: Malignant')
    else: print('  Final Fusion Prediction: Benign')




Now classifying sample: 72_1
  Final Fusion Prediction: Benign
